<h1>Submission prediction</h1>

In [1]:
import pickle
import pandas as pd
import os

PATH_TO_DATA = os.path.join('..', 'data')
PATH_TO_MODELS = os.path.join('..', 'models')

In [2]:
test = pd.read_parquet(os.path.join(PATH_TO_DATA, 'test.parquet'))
test.head()

,id,dates,values
0,6125,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[1.85, -0.04, 0.19, -0.45, -0.75, -0.95, -2.91..."
1,26781,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-0.41, 0.39, -0.47, -0.9, -1.46, -0.51, 0.51,..."
2,13333,"[2016-06-01, 2016-07-01, 2016-08-01, 2016-09-0...","[-0.29, -1.26, 0.17, -1.22, 0.45, -0.94, 0.16,..."
3,53218,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-1.47, 1.55, -0.03, 0.57, -0.57, 0.6, 0.27, 1..."
4,84204,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[2.33, 1.39, -1.03, -2.64, 1.89, 1.77, 1.43, 1..."


<h3>Generating features</h3>

In [3]:
os.chdir('..')
from preprocess import generate_features

In [4]:
features = generate_features(test)
features

c:\Users\sonyi\Documents\GitHub\vk_predict\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\sonyi\Documents\GitHub\vk_predict\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,mean,median,zeros,quantile_25,quantile_75,quantile_05,quantile_95,most_ac_lag,ewm_12,ewmstd_12,dom_freq,mean_d,mean_1y
0,0.037895,-0.010,0.368421,-0.950000,1.0100,-2.542000,2.496000,12,-0.538485,1.285371,0.087719,-0.037857,0.059167
1,-0.016909,-0.080,0.418182,-0.950000,0.5150,-1.731000,2.254000,11,-0.328161,1.479608,0.090909,0.051667,-0.010833
2,-0.039569,-0.160,0.381818,-0.785000,0.5400,-1.316000,1.603466,2,0.411742,1.105920,0.018182,0.031071,-0.412500
3,0.159231,0.110,0.487179,-0.710000,0.7200,-1.977500,2.503000,2,-0.357609,1.366223,0.089744,0.010260,0.252500
4,-0.094705,-0.090,0.476923,-1.030000,0.9100,-2.245463,1.888000,1,-0.902349,0.792104,0.046154,-0.059802,0.995000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.022217,-0.275,0.280488,-0.767500,0.4550,-1.208000,1.916500,1,0.125921,1.000939,0.012195,-0.043131,0.946667
19996,0.072439,0.260,0.292683,-1.010000,1.2600,-2.425500,2.018000,1,-0.058832,1.594123,0.085366,0.010988,-0.167500
19997,0.068727,0.290,0.218182,-0.235000,0.7700,-2.392000,1.604000,1,0.575994,0.720514,0.018182,0.030370,0.738333
19998,-0.037175,-0.180,0.413043,-0.958756,0.9375,-1.580000,1.935000,12,-1.089427,0.921030,0.021739,-0.068482,1.040833


Значения roc auc у Random forest и у Xgboost были равны, но скорость выполнения у последнего была выше. Поэтому используем Xgboost classifier.

In [6]:
os.chdir('notebooks')

In [10]:
xgb_model = pickle.load(open(os.path.join(PATH_TO_MODELS, 'best_xgb.pkl'), 'rb'))
probs = xgb_model.predict_proba(features)[:, 1]

In [19]:
results = pd.DataFrame(columns=['id', 'score'])
results['id'] = test['id']
results['score'] = probs
results

,id,score
0,6125,0.149785
1,26781,0.241258
2,13333,0.321256
3,53218,0.031473
4,84204,0.537314
...,...,...
19995,80341,0.282330
19996,5891,0.012191
19997,29091,0.005924
19998,85877,0.321947


In [21]:
results.to_csv(os.path.join('..', 'submission.csv'), index=False)